# gmplotでヒートマップ

## ライブラリをインポート

In [1]:
from gmplot import gmplot
import sys

import numpy as np
import pandas as pd

from pygeocoder import Geocoder
import googlemaps

sys.path.append("/home/hiroki/Program/Python/GoogleMap/script")
import setting

## APIkeyを読み込む

In [2]:
api_key = setting.API_KEY # API_KEY

## データを読み込む

In [3]:
df = pd.read_csv("./data/001_00.csv",  encoding="shift-jis")

In [4]:
df[15:]

,1,平成27年国勢調査人口速報集計,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
15,17,NaN,01100,1,札幌市,1953784,1913545,40239,2.1028509912,1121.26,1742.5,919198,885848,33350,3.7647542242
16,18,NaN,01101,0,札幌市 中央区,237784,220189,17595,7.9908623955,46.42,5122.4,131598,120741,10857,8.9919745571
17,19,NaN,01102,0,札幌市 北区,285432,278781,6651,2.3857436482,63.57,4490.0,133522,127440,6082,4.7724419335
18,20,NaN,01103,0,札幌市 東区,262075,255873,6202,2.4238587112,56.97,4600.2,124424,118939,5485,4.6116076308
19,21,NaN,01104,0,札幌市 白石区,209770,204259,5511,2.6980451290,34.47,6085.6,103548,100444,3104,3.0902791605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,2393,NaN,47370,c,宮古郡,1196,1231,-35,-2.8432168968,21.99,54.4,468,499,-31,-6.2124248497
2392,2394,NaN,47375,3,多良間村,1196,1231,-35,-2.8432168968,21.99,54.4,468,499,-31,-6.2124248497
2393,2395,NaN,47380,c,八重山郡,5838,5516,322,5.8375634518,363.35,16.1,3196,2713,483,17.8031699226
2394,2396,NaN,47381,3,竹富町,3995,3859,136,3.5242290749,334.39,11.9,2116,2000,116,5.8000000000


## 都市と人口のデータだけ抜き出し

In [5]:
# 都市と人口の抜き出し
pop_df = df.loc[15:,['Unnamed: 4', 'Unnamed: 5']].reset_index()

In [6]:
pop_df

,index,Unnamed: 4,Unnamed: 5
0,15,札幌市,1953784
1,16,札幌市 中央区,237784
2,17,札幌市 北区,285432
3,18,札幌市 東区,262075
4,19,札幌市 白石区,209770
...,...,...,...
2376,2391,宮古郡,1196
2377,2392,多良間村,1196
2378,2393,八重山郡,5838
2379,2394,竹富町,3995


## 政令指定都市のデータだけを抜き出し

In [7]:
gv_city = ['札幌市', '仙台市', 'さいたま市', '千葉市', '横浜市', '川崎市', '相模原市', '新潟市', '静岡市', '浜松市', '名古屋市', '京都市', '大阪市', '堺市', '神戸市', '岡山市', '広島市', '北九州市', '福岡市', '熊本市']

In [8]:
city_df = pop_df[pop_df["Unnamed: 4"].isin(gv_city)].reset_index()

In [9]:
city_df

,level_0,index,Unnamed: 4,Unnamed: 5
0,0,15,札幌市,1953784
1,303,318,仙台市,1082185
2,643,658,さいたま市,1264253
3,727,742,千葉市,972639
4,867,882,横浜市,3726167
5,886,901,川崎市,1475300
6,894,909,相模原市,720914
7,937,952,新潟市,810514
8,1244,1259,静岡市,705238
9,1248,1263,浜松市,798252


## 重みの抜き出し

In [10]:
weight = (city_df["Unnamed: 5"].values).astype('int64')

In [11]:
type(weight)

numpy.ndarray

In [12]:
weight

array([1953784, 1082185, 1264253,  972639, 3726167, 1475300,  720914,
        810514,  705238,  798252, 2296014, 1474570, 2691742,  839891,
       1537860,  719584, 1194507,  961815, 1538510,  741115])

## 重みの正規化

In [13]:
weight_re = (weight - weight.min())

In [14]:
weight_re = weight_re / weight_re.max()

In [15]:
weight_re

array([0.41329869, 0.1247785 , 0.18504738, 0.08851615, 1.        ,
       0.254909  , 0.00518913, 0.03484888, 0.        , 0.03078987,
       0.52658503, 0.25466736, 0.6575805 , 0.04457337, 0.27561786,
       0.00474887, 0.16195978, 0.08493314, 0.27583303, 0.01187615])

In [16]:
weight_re = (weight_re * 10000).astype('int64')
weight_re

array([ 4132,  1247,  1850,   885, 10000,  2549,    51,   348,     0,
         307,  5265,  2546,  6575,   445,  2756,    47,  1619,   849,
        2758,   118])

## 各市の位置情報(緯度・経度)取得

In [17]:
gmaps = googlemaps.Client(key=api_key)

In [18]:
locations = city_df["Unnamed: 4"].values
locations

array(['札幌市', '仙台市', 'さいたま市', '千葉市', '横浜市', '川崎市', '相模原市', '新潟市', '静岡市',
       '浜松市', '名古屋市', '京都市', '大阪市', '堺市', '神戸市', '岡山市', '広島市', '北九州市',
       '福岡市', '熊本市'], dtype=object)

In [19]:
results = []
for address in locations:
    print(address)
    result = gmaps.geocode(address)
    lat = result[0]["geometry"]["location"]["lat"]
    lon = result[0]["geometry"]["location"]["lng"]
    print(lat, lon)
    results.append([lat,lon])
results = np.array(results)

札幌市
43.0617713 141.3544507
仙台市
38.268195 140.869418
さいたま市
35.8616486 139.6454782
千葉市
35.6074041 140.1065366
横浜市
35.4436739 139.6379639
川崎市
35.5308325 139.7029125
相模原市
35.5712568 139.373427
新潟市
37.9161244 139.0363708
静岡市
34.9755668 138.3826773
浜松市
34.7102752 137.7274159
名古屋市
35.18145060000001 136.9065571
京都市
35.011564 135.7681489
大阪市
34.6937249 135.5022535
堺市
34.5733261 135.4831176
神戸市
34.6900806 135.1956311
岡山市
34.6555312 133.919795
広島市
34.3852894 132.4553055
北九州市
33.8834976 130.8751773
福岡市
33.5901838 130.4016888
熊本市
32.8032164 130.7079369


In [20]:
results[:,0]

array([43.0617713, 38.268195 , 35.8616486, 35.6074041, 35.4436739,
       35.5308325, 35.5712568, 37.9161244, 34.9755668, 34.7102752,
       35.1814506, 35.011564 , 34.6937249, 34.5733261, 34.6900806,
       34.6555312, 34.3852894, 33.8834976, 33.5901838, 32.8032164])

## マップの中心地を指定

In [21]:
gmap = gmplot.GoogleMapPlotter(38.226198483664376, 141.0229438945786, 5,apikey=str(api_key))

In [24]:
gmap.heatmap(results[:, 0], results[:, 1], weights=weight_re, radius=50, maxIntensity=weight_re.max())

In [25]:
gmap.draw("population_japan.html")